<a href="https://colab.research.google.com/github/dlsun/pods/blob/master/03-Quantitative-Data/3_1_Simple_Analyses_of_Quantitative_Variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the previous chapter, we discussed ways to analyze categorical variables, such as **gender**, **class**, and **embarked** in the Titanic data set. In this chapter, we discuss ways to analyze quantitative variables, such as **age** and **fare**.

In [0]:
import pandas as pd
data_dir = "http://dlsun.github.io/pods/data/"
df_titanic = pd.read_csv(data_dir + "titanic.csv")
df_titanic

# Simple Analyses of Quantitative Variables

We begin by discussing ways to summarize and visualize a single quantitative variable.

If a column in a `pandas` `DataFrame` has a numeric type, such as `int64` or `float64`, then the `.describe()` method will automatically produce many of the summaries discussed in this lesson.

In [0]:
df_titanic["age"].describe()

## Summarizing a Quantitative Variable

What statistics should we use to summarize a quantitative variable? The most salient features of a quantitative variable are its center and spread.

### Measures of Center

Some statistics measure the _center_ of a variable. Two commonly used measures of the center are:

- the _mean_ (a.k.a. average): the sum of the values divided by the count. If the values are represented by the variables $x_1, x_2, ..., x_n$, then the mean $\bar x$ is defined as 
$$ \bar x = \frac{1}{n} \sum_{i=1}^n x_i. $$
- the _median_: the middle value when you sort the values (i.e., a value such that 50% of the values lie below and 50% of the values lie above)

A measure of center gives us information about the "typical" value of a variable. For example, you might not know whether a typical fare on the Titanic was £1, £10, or £100. But if we calculate the mean:

In [0]:
df_titanic.fare.mean()

we see that a typical fare is around £30.

Let's see what the median says about the "typical" fare:

In [0]:
df_titanic.fare.median()

The median is quite different from the mean! It says that about 50% of the passengers paid less than £15 and about 50% paid more, so another reasonable value for the "typical" fare is £15.

The mean was twice the median! What explains this discrepancy? The reason is that the mean is very sensitive to extreme values. To see this, let's look at the highest fare that any passenger paid.

In [0]:
df_titanic.fare.max()

The highest fare paid was over £500! Even if most passengers paid less than £15, extreme values like this one will pull the mean upward. On the other hand, since the median is always the middle value, it is not affected by the extreme values, as long as the ordering of the values does not change.

To drive this point home, let's see what would happen to the mean and median if that maximum fare were (hypothetically) £10,000.

In [0]:
fare_10k = df_titanic.fare.replace(df_titanic.fare.max(), 10000)
fare_10k.mean(), fare_10k.median()

Notice how the mean now increases to over £60, but the median is unaffected.

Just to satisfy our curiosity, let's learn more about this passenger who paid the maximum fare. To do this, we have to find the row that achieved this maximum value. Fortunately, there is a convenient `pandas` function, `.idxmax()`, that returns the _row index_ of the maximum fare. (A mathematician might call this the ["arg max"](https://en.wikipedia.org/wiki/Arg_max).)

In [0]:
df_titanic.fare.idxmax()

Now we can select the row corresponding to this index using `.loc`, as we learned in the previous section.

In [0]:
df_titanic.loc[df_titanic.fare.idxmax()]

The median is a number below which 50% of the values fall. What if we want to know some other percentile? We can use the `.quantile()` function, which takes a percentile rank (between 0 and 1) as input and returns the corresponding percentile.

For example, the 75th percentile is:

In [0]:
df_titanic.fare.quantile(.75)

which is pretty close to the mean. So only about 25% of the passengers paid more than the mean! The mean is not a great measure of center when there are extreme values, as in this data set.

To summarize, we have encountered several `pandas` functions that can be used to summarize a quantitative variable:

- `.mean()` calculates the mean or average.
- `.median()` calculates the median.
- `.quantile(q)` returns a value such that a fraction `q` of the values fall below that value (in other words, the (100q)th percentile).
- `.max()` calculates the maximum value.
- `.idxmax()` returns the index of the row with the maximum value. If there are multiple rows that achieve this value, then it will only return the index of the first occurrence.

The corresponding functions for the _minimum_ value exist as well:

- `.min()` calculates the minimum value.
- `.idxmin()` returns the index of the row with the minimum value. If there are multiple rows that achieve this value, then it will only return the index of the first occurrence.

In [0]:
df_titanic.fare.min()

Some passengers boarded the Titanic for free, apparently.

### Measures of Spread

The center of a quantitative variable only tells part of the story. For one, it tells us nothing about how spread out the values are. Therefore, it is important to also report a measure of _spread_. The measures of spread that we will explore in this section are:
- mean absolute deviation
- variance
- standard deviation



Let's investigate a few measures of spread that are built into `pandas`. For completeness, the formulas for these statistics are provided, where $x_1, ..., x_n$ represent the values and $\bar x$ their mean. But the intuition is more important to understand than the formulas.

The first statistic that might come to mind is the _mean absolute deviation_, or _MAD_. To calculate the MAD, we first calculate the difference between each observation and the mean. Values below the mean will have a negative difference, while values above the mean will have a positive difference. We don't want the negative differences to cancel out the positive differences, since _all_ of them contribute to the spread. So we take the absolute value of all the differences and then average.

$$
\begin{align*}
\textrm{MAD} &= \textrm{mean of } |x_i - \bar x| \\
&= \frac{1}{n} \sum_{i=1}^n |x_i - \bar x|
\end{align*}
$$

We can implement the MAD ourselves using the `.mean()` and `.abs()` functions.

In [0]:
# STEP 1: Calculate the difference between each fare and the mean.
(df_titanic.fare - df_titanic.fare.mean())

In [0]:
# STEP 2: Calculate the absolute value of each difference.
(df_titanic.fare - df_titanic.fare.mean()).abs()

In [0]:
# STEP 3: Take the mean of these absolute differences.
(df_titanic.fare - df_titanic.fare.mean()).abs().mean()

Notice that in Step 1, we subtracted a single number (`df_titanic.fare.mean()`) from a `pandas` `Series` (`df_titanic.fare`). `pandas`  automatically _broadcasted_ the subtraction over each number in the `Series`.

The `.abs()` function in Step 2 is an example of a _vectorized_ function. The absolute value function is applied to each element of the `Series`.

The MAD is actually built into `pandas`, so there really is no reason to implement it from scratch, as we did above. Let's check that we get the same answer when we call the built-in function.

In [0]:
df_titanic.fare.mad()

Since the MAD is a mean of the absolute differences and the mean represents the "typical" value, we can interpret the MAD as saying that the "typical" fare is about £30 away from the average.

A different way to ensure that the negative and positive differences don't cancel is to square all the differences before averaging. This leads to the definition of _variance_.

\begin{align}
\textrm{Variance} &= \textrm{mean of } (x_i - \bar x)^2 \\
&= \frac{1}{n} \sum_{i=1}^n (x_i - \bar x)^2
\end{align}

We can implement the variance ourselves. Again, notice how the subtraction and the power operations are broadcast over the elements of the `Series`.

In [0]:
((df_titanic.fare - df_titanic.fare.mean()) ** 2).mean()

Alternatively, we can simply call the `.var()` function in `pandas`.

In [0]:
df_titanic.fare.var()

You might be surprised that `.var()` produced a slightly different number. This is because `pandas` divided by $n-1$ (rather than $n$) in calculating the mean of the squared differences. That is, the default variance formula that `pandas` uses is 

$$\text{Variance} = \frac{1}{n-1} \sum_{i=1}^n (x_i - \bar x)^2.$$

To force Pandas to divide by $n$, you can set `ddof=0`.

In [0]:
df_titanic.fare.var(ddof=0)

The trouble with variance is that its units are wrong. If the original values $x_1, ..., x_n$ are in £, the variance would be in squared £. This is obvious if you simply look at the magnitude of the variance in the example above; the variance is in the _thousands_, even though the largest fare is just over £500.  To correct the units of variance, we take the square root to obtain a more interpretable measure of spread, called the _standard deviation_:

$$\textrm{SD} = \sqrt{\textrm{Variance}}.$$

In [0]:
df_titanic.fare.std()

This standard deviation has a clean interpretation: the "typical" fare is about £50 away from the average.

The standard deviation is the most widely used measure of spread, more common than the MAD. At first, this might seem odd. To calculate the standard deviation, we squared the differences from the mean, only to take a square root in the end. Why bother with this rigmarole, when we could just calculate absolute values instead?

The reasons for preferring the standard deviation are complicated. But the short answer is that the variance (i.e., the square of the standard deviation) is much nicer mathematically. If you know calculus, you might remember that the absolute value function does not have a derivative at 0. Therefore, the MAD is not _differentiable_, which makes it inconvenient mathematically. That doesn't necessarily mean that it's any worse as a measure of spread.

## Visualizing a Quantitative Variable

Graphics can help us understand how the values of a quantitative variable are distributed. We will study two types of visualizations for quantitative variables: histograms and densities.

### Histograms

The standard visualization for a single quantitative variable is the _histogram_. A histogram sorts the values into bins and uses bars to represent the number of values in each bin.

To make a histogram, we call the `.plot.hist()` method of the selected variable.

In [0]:
df_titanic.fare.plot.hist()

It seems that we did not get enough resolution to really determine what is going on at the lower end of the scale. Let's ask for more bins.

In [0]:
df_titanic.fare.plot.hist(bins=50)

From this graph, we see a concentration of values around 10-30 (which we previously identified as the "center") and a spread of about 30-50 (which we previously identified as the "spread"). We also see the outlier who paid more than £500. We also see features that were not obvious before: the skewed shape of the distribution, the gap between £300 and £500, and so on. This single picture has managed to convey more information than a dozen summary statistics.

We might want to plot multiple histograms on the same graphic to make for easy comparison. To do this, we simply make multiple calls to plotting functions within the same cell. For example, if we wanted to compare the distributions of the ages of 1st class and 3rd class passengers, we could call `.plot.hist()` twice.

In [0]:
df_titanic[df_titanic["class"] == "1st"]["age"].plot.hist()
df_titanic[df_titanic["class"] == "3rd"]["age"].plot.hist()

Notice that `pandas` automatically plotted the two histograms using different colors. There are two problems with this plot. First, we don't know which color corresponds to which variable. Second, we cannot see the blue histogram underneath the orange histogram because the colors are opaque.

To solve the first problem, we add a legend for each variable by specifying `legend=True`. The label that will be used in the legend will be the name of the `Series` by default; to customize this, we can specify the label using `label=`.

To solve the second problem, we set the transparency `alpha`, which is a number between 0 and 1, with 0 being perfectly transparent and 1 being completely opaque. Try varying `alpha` to get a feel for what it does.

In [0]:
df_titanic[df_titanic["class"] == "1st"]["age"].plot.hist(
    label="1st", legend=True, alpha=0.5
)
df_titanic[df_titanic["class"] == "3rd"]["age"].plot.hist(
    label="3rd", legend=True, alpha=0.5
)

### Densities

Another way to visualize the distribution of a quantitative variable is by plotting its **density**. A density plot turns the jagged histogram into a smooth curve, allowing the user to focus on the general shape of the distribution.

In [0]:
df_titanic.fare.plot.density()

The x-axis is too wide. (You cannot have negative fares.) We can set the limits of the x-axis manually using the `xlim` argument.

In [0]:
df_titanic.fare.plot.density(xlim=(0, 600))

Let's superimpose this density curve on top of the histogram, by making two calls to plotting functions:

In [0]:
df_titanic.fare.plot.hist(bins=50)
df_titanic.fare.plot.density(xlim=(0, 600))

If you squint at this plot, you will see an orange line at the bottom of the plot. This is supposed to be the density. But why does it appear as a flat line? The y-axis offers a hint. When we made the density plot earlier, the y-axis extended from 0 to about 0.02. Now the y-axis extends all the way to 500. On such a scale, a curve that fluctuates between 0 and 0.02 will appear to be a flat line!

The problem is that the histogram and the density are currently on different scales. By default, histograms display counts, while densities display distributions. In other words, densities are normalized so that the total area under the curve is 1. To be able to display a histogram and density on the same graph, we have to normalize the histogram so that the total area of the bars is 1. We can do this by setting the option `density=True`.

In [0]:
df_titanic.fare.plot.hist(bins=50, density=True)
df_titanic.fare.plot.density(xlim=(0, 600))

Now we can clearly see how the density smooths the histogram. It does a pretty good job for the most part, but it "oversmooths" near 0, missing the spike.

You might wonder what options are available besides `legend`, `alpha`, `bins`, `density`, and `xlim`. Let's look at the documentation for `.plot.hist()`.

In [0]:
df_titanic.fare.plot.hist?

The documentation for `.plot.hist()` tells us about one option, `bins`, but then refers us to `.plot()` for additional options. If you peek inside `.plot()`, you will find the `legend`, `density`, and `xlim` options, as well as many others.

In [0]:
df_titanic.fare.plot?

Try these out to get a feel for what they do. For example, can you figure out how to make the density curve red in the above graph? Or how to change the size of the figure?

You might be surprised that `alpha` is not listed among the options above. This is because `alpha` is an option of the `matplotlib` plotting method that `pandas` calls. (Notice that the last argument of `.plot()` is `**kwds`, for "options to pass to matplotlib plotting method".)

## Exercises

Exercises 1-6 ask you to work with the tips data set (https://dlsun.github.io/pods/data/tips.csv), which contains data on the tips paid to a waiter by different parties.

1\. How many people were in the largest party served by the waiter? The smallest?

2\. How could you use the `.quantile()` function to calculate the median? Check that your method works on an appropriate variable from the Tips data set.

3\. Another measure of spread is the **interquartile range**, or IQR, defined as:

$$ \textrm{IQR} = \textrm{75th percentile} - \textrm{25th percentile}. $$

Measure the spread in the total bills by reporting the IQR.

4\. Some people use MAD to refer to the **median absolute deviation**. The median absolute deviation is the same as the mean absolute deviation, but it uses the median instead of the mean:

$$\textrm{M(edian)AD} = \textrm{median of } |x_i - \textrm{median}|. $$

Calculate the median absolute deviation of the total bills. (The median absolute deviation is not built into Pandas, so you will have to implement it from scratch.)

5\. Make a graphic that shows the distribution of total bills. Explain what you see.

6\. Recall from the previous chapter that _bar charts_ are used to visualize categorical variables. What happens when you call `.plot.bar()` on a quantitative variable, like `tip`? Why does it look this way? Do you think bar charts are appropriate for quantitative variables?